# Modeling Electric Field of a Moving Charge
We know from chapter 3 that there are two contributing components to an electric field, the velocity and the acceleration fields, such that
$$
E_{total} = E_{vel} + E_{acc}
$$

Recal that  these fields take the form

$$
E_{vel} = q\left[\frac{\left(\vec{n}-\vec{\beta}\right)\left(1-\vec{\beta}^{2}\right)}{\kappa^{3}R^{2}}\right]
$$
and
$$
E_{acc} = \frac{q}{c}\left[\frac{\vec{n}}{\kappa^{3}R}\times\left(\left(\vec{n}-\vec{\beta}\right)\times \dot{\vec{\beta}}\right)\right]
$$

As you will see in Chapter 3.3 there are limiting cases we can take when $\beta << c$. However, since we have the tools of numeric simulations at our disposal we can also peak ahead to some future chapters and see what the effects of this full formulation is on the electric field with a moving and an accelerating charge.

In order to run this notebook you will need tqdm and mplEasyAnimate installed

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from mplEasyAnimate import animation
from IPython.display import Video
plt.style.use(pubStyle)

Below is a very simple (and slap-dash) approach to simulating this situation. I first setup initial conditions, then I model the particle moving along the x axis with a period of acceleration (from taccel to t=1). Next I calculate the electric field at every point on a grid considering the retarded time

In [198]:
# Constants
c = 1 # Speed of light (m/s)
epsilon_0 = 1 # Vacuum permittivity (F/m)
q = 1  # Charge in Coulombs (adjust as needed)

# Initial conditions
r0 = np.array([-1.0, 0.0])   # Initial position (m)
drdt = np.array([0.8, 0.0])  # Initial velocity (m/s)
dvdt = np.array([0.0, 0.0])  # Initial acceleration (m/s^2)
tmax = 2
taccel = 0.5
dt = 0.005
times = np.arange(0, tmax, dt)
num_steps = len(times)

# Arrays to store position, velocity, and acceleration
rHist = np.zeros((num_steps, 2))
vHist = np.zeros((num_steps, 2))
aHist = np.zeros((num_steps, 2))

# Initial position and velocity
r = r0.copy()
v = drdt.copy()
a = dvdt.copy()

## Model the Particle Position
for tidx, t in enumerate(times):
    if t >= taccel and np.sqrt(v[0]**2 + v[1]**2) >= 0:
        dvdt = np.array([-1, 0.0])
    else:
        dvdt = np.array([0, 0])

    # Update acceleration, velocity, and position
    a = dvdt
    v += a * dt
    r += v * dt

    # Store values
    rHist[tidx] = r
    vHist[tidx] = v
    aHist[tidx] = a


# Spatial grid for field calculation
x = np.linspace(-2, 2, 50)
y = np.linspace(-2, 2, 50)
X, Y = np.meshgrid(x, y)
grid_shape = X.shape

# Initialize electric field history array
EHist = np.zeros((num_steps, 2, *grid_shape))

# Precompute field point positions
field_points = np.stack((X.flatten(), Y.flatten()), axis=-1)

# Loop over time steps
for tidx, t in tqdm(enumerate(times), total=num_steps):
    # Charge's current position and velocity
    r_q_t = rHist[tidx]
    v_q_t = vHist[tidx]
    a_q_t = aHist[tidx]

    # Initialize arrays for this time step
    Ex = np.zeros(grid_shape)
    Ey = np.zeros(grid_shape)

    for idx, point in enumerate(field_points):
        i, j = np.unravel_index(idx, grid_shape)
        
        # Approximate retarded time
        R = point - r_q_t
        R_mag = np.linalg.norm(R)
        t_ret = t - R_mag / c

        # Find the index closest to t_ret since it is possible that there was not a simulation step at exatly the retarded time
        t_ret_idx = np.argmin(np.abs(times - t_ret))

        # Get the charge's position, velocity, and acceleration at retarded time
        r_q_tr = rHist[t_ret_idx]
        v_q_tr = vHist[t_ret_idx]
        a_q_tr = aHist[t_ret_idx]

        # Recalculate R and R_mag using retarded position
        R = point - r_q_tr
        R_mag = np.linalg.norm(R)

        # Avoid division by zero
        if R_mag < 1e-6:
            continue

        # **Extend n and beta to 3D by adding zero z-component** This is to make the cross product work properly since it is not well defined in 2D
        n = R / R_mag  # Unit vector from charge to field point
        n_3d = np.append(n, 0)

        beta = v_q_tr / c
        beta_3d = np.append(beta, 0)

        beta_dot = a_q_tr / c
        beta_dot_3d = np.append(beta_dot, 0)

        # Scalar product n · beta
        n_dot_beta = np.dot(n, beta)
        gamma = 1 / np.sqrt(1 - np.linalg.norm(beta)**2)

        # Denominator term
        denom = (1 - n_dot_beta)**3 * R_mag**2

        # **First term (velocity term) using 3D vectors**
        term1 = (n_3d - beta_3d) / denom

        # Compute the cross products in 3D
        cross_inner = np.cross(n_3d - beta_3d, beta_dot_3d)
        cross = np.cross(n_3d, cross_inner)

        # Second term (acceleration term)
        term2 = cross / ((1 - n_dot_beta)**3 * R_mag * c)

        # Electric field vector in 3D
        E_3d = (q / (4 * np.pi * epsilon_0)) * (term1 + term2)

        # Since we are in 2D, extract the x and y components
        E = E_3d[:2]

        # Store components
        Ex[i, j] = E[0]
        Ey[i, j] = E[1]

    # Store electric field for this time step
    EHist[tidx, 0] = Ex
    EHist[tidx, 1] = Ey

  0%|          | 0/400 [00:00<?, ?it/s]

Now we can visualize the simulation. There are many ways in which this could be done; however, one of the most intuitive is to make a movie of what is happening. I developed and maintain a package called mplEasyAnimate which makes making movies from matplotlib figures trivial. 

The idea here is that we will add a frame for each timestep in the simulation. Assuming most of the frame looks the same and just the data changes from timestep to timestep this will produce a nice animation of what we simulated. 

We next need to ask what kind of visualization would be right for a single timestep. There are a few resonable ways to vizualize an electric field; however, since we are interested in the field line density the most obvious is a contour plot which will show lines of constant electric field strength. In a contour plot the more densly packed lines are the larger the gradient (i.e the faster that parameter is varying)

In [199]:
with animation("RetardedPotential-LowResolution.mp4", fps=60, dpi=50) as anim:
    for EField, r in tqdm(zip(EHist, rHist), total=num_steps):
        E_magnitude = np.sqrt(EField[0]**2 + EField[1]**2)
        Ex_norm = EField[0] / E_magnitude
        Ey_norm = EField[1] / E_magnitude
        fig, ax = plt.subplots(figsize=(8, 8))
        
        # Contour plot of electric field magnitude
        contour = ax.contour(X, Y, E_magnitude, levels=30, cmap='inferno', alpha=0.7)
        # ax.pcolormesh(X, Y, E_magnitude)
        ax.plot(r[0], r[1], 'ro', markersize=3)
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        anim.add_frame(fig)

        # We need to close the figure so that we dont fill up memory with frames already rendered out
        plt.close(fig)


  0%|          | 0/400 [00:00<?, ?it/s]

In [202]:
Video("RetardedPotential-LowResolution.mp4")